# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove the `raise NotImplementedError()` statement.

The first two questions are on phase estimation and the rest is about order finding algorithm.

## Question 1 (2 points)

Complete the function `create_operator` that returns a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$.

The function `create_operator` has

- Input: $\phi$
- Returns: Controlled $U$ gate with the properties described above

Don't create any circuit. Only a gate should be returned.

In [2]:
from cirq import CZPowGate

def create_operator(phi):
    # YOUR CODE HERE
    CU = CZPowGate(exponent=phi*2)
    
    # Do not modify anything below this line  
    return CU

In [3]:
# You can use this code to test your function by checking the bottom right corner of the unitary matrix
import cirq
from cmath import exp, pi

def test(phi):
    CU = create_operator(phi)
    unitary_matrix = cirq.unitary(CU)
    print(unitary_matrix)

phi=3/5
test(phi)
print(exp(2*pi*phi*1j))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.80901699-0.58778525j]]
(-0.8090169943749476-0.587785252292473j)


In [ ]:
# hidden tests in this cell will be used for grading.

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (8 points)

Complete the function `estimate_phi` such that given a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$, it estimates and returns $\phi$.

The function `estimate_phi` has

- Input: Controlled $U$ gate
- Returns: Estimate for phi 

You are given iqft and qpe algorithms and you can use them in your solution. Let the size of the first register equal 10.

In [4]:
def iqft(n,qubits,circuit):
    
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)
     
    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step
            
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):
    
    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))
    
    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate 
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))
        
    #Apply inverse QFT
    iqft(t,control,circuit)

In [5]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def estimate_phi(mystery):
    

    #Create cirucit
    circuit = cirq.Circuit()
    # YOUR CODE HERE
    n = 1
    t = 10
    
    control = [cirq.LineQubit(i) for i in range(t) ]
    
    target = [cirq.LineQubit(i) for i in range(t,t+n) ]
    circuit.append(cirq.X.on_each(target))

    qpe(t,control, target, circuit, mystery)
    
    circuit.append(cirq.measure(*control, key='result'))
    
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)
    
    #Most frequent observation?
    freq = list(samples.histogram(key='result').keys())[0]
    
    return freq/2**t

In [6]:
import math
# You can use this code to test your function by different operators
def test_qpe(phi):
    operator = CZPowGate(exponent=2*phi)
    return estimate_phi(operator)

assert(math.isclose(test_qpe(0.23),0.23,rel_tol=1e-2))

In [ ]:
# hidden tests will be used for grading.

## Question 3 (1 point)

Complete the function `calcL` such that given $N$, it returns the size $L$ of the target register. Note that $L = \log_2 N$. 

The function `calcL` has

- Input: $N$
- Returns: size of the target register

In [7]:
import math
def calcL(N):
    # YOUR CODE HERE
    L=math.ceil(math.log2(N))
    return L

In [8]:
# You can use this cell to test your function

print(calcL(30))
print(calcL(62))

5
6


In [ ]:
# hidden tests will be used for grading.

## Question 4 (1 point)

Complete the function `calct` such that given $N$, it returns the size $t$ of the control register. Note that $t = 2L + 1 + \big \lceil \log \big( 2 + \frac{1}{2\epsilon} \big) \big \rceil$. Let $\epsilon=0.1$. 

The function `calct` has

- Input: $N$
- Returns: size of the control register

In [9]:
import math
def calct(N):
    # YOUR CODE HERE
    L = calcL(N)
    e = 0.1
    t = 2*L + 1 + math.ceil(math.log2(2 + 1/(2*e)))
    return t

In [10]:
# You can use this cell to test your function

print(calct(30)) #For 30 the answer is 14
print(calct(62)) #For 62 the answer is 16

14
16


In [11]:
# hidden tests will be used for grading.

## Question 5 ( 2 points)

Complete the function `create_regs` such that given the size of the control and target registers, it returns the control and target registers.

The function `create_regs` has

- Input: $L$ and $t$
- Returns: Two registers `control` and `target`

In [12]:
import cirq
def create_regs(t,L):

    # YOUR CODE HERE
    n = L
    
    control = [cirq.LineQubit(i) for i in range(1,t+1) ]
    
    target = [cirq.LineQubit(i) for i in range(t+1,t+n+1) ]
    
    return control, target

In [13]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect

In [14]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect

## Question 6 (8 points)

Complete the function `order` so that given $x$ and $N$, it returns the histogram of samples from the order finding circuit.

The function `order` has

- Input: $x$ and $N$
- Returns: A histogram of samples

Notes:

- You are given a function named $U_x$ which implements $U_x |y\rangle \rightarrow |xy {\mod N}\rangle $ and returns its controlled version. Run the following cell to load the function. In order to use the function you should pass $x$ and $N$ as parameter.

    <pre>CU=Ux(x,N)</pre>

- You are also given the code for `qpe` and `iqft`.

In [15]:
import numpy as np
def Ux(x,N):

    k=1
    while(N>2**k):
        k=k+1
        
    u = np.zeros([2**k, 2**k], dtype = int) 

    for i in range(N):
        u[x*i%N][i]=1
    for i in range(N,2**k):
        u[i][i]=1
        

    XU = cirq.MatrixGate(u).controlled()
    return XU

In [16]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def iqft(n,qubits,circuit):
    
    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)
     
    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step
            
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):
    
    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))
    
    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate 
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))
        
    #Apply inverse QFT
    iqft(t,control,circuit)

In [17]:
import cirq
from cirq import X, SWAP

def order(x,N):
    #Create a circuit
    circuit = cirq.Circuit()
    # YOUR CODE HERE
    CU=Ux(x,N)
    
    L = calcL(N)
    n = L
    t = calct(N)
    
    control, target = create_regs(t,L)
    
    circuit.append(X(target[n-1]))
    
    qpe(t,control, target, circuit, CU)

    #Measure the control register
    circuit.append(cirq.measure(*control, key='result'))

    #Sample the circuit
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    # Return a histogram of samples
    results= samples.histogram(key='result')
    return results

In [18]:
#You can use this cell to test your function

results = order(4,31)
print(results)

Counter({13107: 180, 0: 180, 3277: 167, 9830: 122, 6554: 96, 6553: 67, 9831: 60, 9829: 13, 3276: 12, 6555: 11, 3278: 10, 6552: 10, 9832: 9, 13106: 8, 13108: 8, 9828: 5, 3275: 4, 9835: 3, 3279: 3, 9834: 2, 9833: 2, 6549: 2, 13105: 2, 6563: 1, 9825: 1, 9841: 1, 3255: 1, 13095: 1, 6556: 1, 6551: 1, 9824: 1, 13111: 1, 6543: 1, 3283: 1, 13110: 1, 6557: 1, 9845: 1, 6550: 1, 13136: 1, 13112: 1, 3269: 1, 6559: 1, 3289: 1, 6560: 1, 6539: 1, 3274: 1, 3280: 1})


In [18]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect